# Full CNN

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle

In [ ]:
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
tf.device('/gpu:0') 

# 读取数据

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)
y = y.reshape(-1,1)
y = Scaler.fit_transform(y)

In [ ]:
X_shuffle, y_shuffle = shuffle(X, y)

In [ ]:
train_X, train_y = X_shuffle[:300,:,:,:], y_shuffle[:300]
test_X, test_y = X_shuffle[300:, :, :, :], y_shuffle[300:]

# 建立模型

In [ ]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(32,(3,3), activation='relu',padding='same', input_shape=(1000,10,6)))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(64,(3,3),padding='same',activation='relu'))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(64,(3,3),padding='same',activation='relu'))
model_1.add(layers.Flatten())
model_1.add(layers.Dropout(0.5))
#model_1.add(layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=-1)))
#model_1.add(layers.LSTM(units=128, return_sequences=True))
model_1.add(layers.Dense(128,activation='relu'))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(1,activation='relu'))

In [ ]:
model_1.compile(optimizer=optimizers.Adam(lr=0.001),
               loss='mse',
               metrics=['mae']
               )

In [ ]:
early_stopping = EarlyStopping(monitor='val_mae', patience=50, verbose=2)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', patience=10, verbose=1, factor=0.5, min_lr=0.00001)
checkpoint = ModelCheckpoint('E:\\毕业设计\\毕业\\V0\\data\model\\full_cnn.h5', monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callback = [early_stopping, learning_rate_reduction, checkpoint]
history = model_1.fit(train_X, train_y, epochs=1000, validation_data=(test_X, test_y), batch_size=10,callbacks=callback)

In [ ]:
plt.plot(history.history['loss'], label='训练集loss', linewidth=2)
plt.plot(history.history['val_loss'], label='验证集loss', linewidth=2)
plt.xlabel('训练轮次')
plt.ylabel('loss')
plt.legend()
plt.grid(linestyle='-.')
plt.savefig('E:\\毕业设计\\毕业\\V0\\figs\\第五章\\loss训练过程.png',dpi=600)

In [ ]:
plt.plot(history.history['mae'], label='训练集mae', linewidth=2)
plt.plot(history.history['val_mae'], label='验证集mae', linewidth=2)
plt.xlabel('训练轮次')
plt.ylabel('MAE/Ah')
plt.legend()
plt.grid(linestyle='-.')
plt.savefig('E:\\毕业设计\\毕业\\V0\\figs\\第五章\\mae训练过程.png',dpi=600)

In [ ]:
res = model_1.predict(X)

In [ ]:
plt.plot(Scaler.inverse_transform(res))
plt.plot(Scaler.inverse_transform(y))

In [ ]:
res = Scaler.inverse_transform(res)
res = pd.DataFrame(res)
res.to_csv('E:\\毕业设计\\毕业\\V0\\data\\容量预测结果_全CNN模型.csv',index=False,encoding="utf_8_sig")

In [ ]:
model_1.save('E:\\毕业设计\\毕业\\V0\\data\model\\full_cnn.h5')

# softmax预测概率分布

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from scipy.signal import savgol_filter

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)

In [ ]:
for i in range(304):
    if i < 100:
        y[i] = 0
    elif i < 230:
        y[i] = 1
    else:
        y[i] = 2

In [ ]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(64,(3,3), activation='relu',padding='same', input_shape=(1000,10,6)))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(64,(3,3),padding='same',activation='relu'))
model_1.add(layers.MaxPooling2D((2,2)))
model_1.add(layers.Conv2D(128,(3,3),padding='same',activation='relu'))
model_1.add(layers.Flatten())
model_1.add(layers.Dropout(0.5))
#model_1.add(layers.Lambda(lambda x: keras.backend.expand_dims(x, axis=-1)))
#model_1.add(layers.LSTM(units=128, return_sequences=True))
model_1.add(layers.Dense(128,activation='relu'))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(3,activation='softmax'))

In [ ]:
model_1.compile(optimizer=optimizers.Adam(lr=0.0001),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy']
               )

In [ ]:
model_1.fit(X,y,epochs=50,batch_size=5)

In [ ]:
res = model_1.predict(X)

In [ ]:
temp = pd.DataFrame(res)
temp.iloc[:,0] = state_1
temp.iloc[:,1] = state_2
temp.iloc[:,2] = state_3
temp.to_csv('E:\\毕业设计\\毕业\\V0\\data\状态概率预测.csv',index=False,encoding="utf_8_sig")

In [ ]:
temp = pd.DataFrame(res)
temp.iloc[:,0] = state_1
temp.iloc[:,1] = state_2
temp.iloc[:,2] = state_3
temp.to_csv('E:\\毕业设计\\毕业\\V0\\data\系统状态概率预测.csv',index=False,encoding="utf_8_sig")

# CNN+LSTM混合模型

混合模型需要训练好长时间才能效果好，至少要几百个轮次，需要配置GPU或用云跑。

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.layers import concatenate, LSTM, Conv1D, Conv2D, Flatten, TimeDistributed, Input, Dense, MaxPooling1D, MaxPooling2D, Dropout, Activation
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)
y = y.reshape(-1,1)
y = Scaler.fit_transform(y)

In [ ]:
X_shuffle, y_shuffle = shuffle(X, y)

In [ ]:
train_X, train_y = X_shuffle[:263,:,:,:], y_shuffle[:263]
test_X, test_y = X_shuffle[263:, :, :, :], y_shuffle[263:]

In [ ]:
#X = np.expand_dims(X, axis=-1)
#X.shape

In [ ]:
model_2 = models.Sequential()

In [ ]:
model_2.add(TimeDistributed(Conv1D(64,9, activation='relu',padding='same', input_shape=(1000,10,6))))

In [ ]:
model_2.add(TimeDistributed(MaxPooling1D()))

In [ ]:
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))

In [ ]:
model_2.add(TimeDistributed(MaxPooling1D()))

In [ ]:
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))

In [ ]:
model_2.add(TimeDistributed(Flatten())) 

In [ ]:
model_2.add(Dropout(0.5)) 

In [ ]:
model_2.add(LSTM(units=128, return_sequences=True))

In [ ]:
model_2.add(LSTM(units=50, return_sequences=False))

In [ ]:
#model.add(Dense(128,activation='relu'))
#model_2.add(Dense(64,activation='relu'))
model_2.add(Dense(1,activation='relu'))

In [ ]:
model_2.compile(optimizer=optimizers.Adam(lr=0.0001),
               loss='mse',
               metrics=['mae'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_mae', patience=50, verbose=2)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', patience=10, verbose=1, factor=0.5, min_lr=0.00001)
checkpoint = ModelCheckpoint('E:\\毕业设计\\毕业\\V0\\data\model\\cnn+lstm.h5', monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callback = [early_stopping, learning_rate_reduction, checkpoint]
history = model_2.fit(train_X, train_y, epochs=1000, validation_data=(test_X, test_y), batch_size=10,callbacks=callback)

In [ ]:
res = model_2.predict(X)

In [ ]:
plt.plot(Scaler.inverse_transform(res))
plt.plot(Scaler.inverse_transform(y))

In [ ]:
plt.plot(Scaler.inverse_transform(res))
plt.plot(Scaler.inverse_transform(y))

In [ ]:
res = Scaler.inverse_transform(res)
res = pd.DataFrame(res)
res.to_csv('E:\\毕业设计\\毕业\\V0\\data\\容量预测结果_混合模型.csv',index=False,encoding="utf_8_sig")

In [ ]:
model_2.save('E:\\毕业设计\\毕业\\V0\\data\model\\cnn+lstm.h5')

# CNN+LSTM+BNN混合模型（得上云跑了）

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.layers import concatenate, LSTM, Conv1D, Conv2D, Flatten, TimeDistributed, Input, Dense, MaxPooling1D, MaxPooling2D, Dropout, Activation, Reshape
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle

In [ ]:
X = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\X.npy')
y = np.load('E:\\毕业设计\\毕业\\V0\\data\\data_processed\\y.npy')

In [ ]:
Scaler = MinMaxScaler()
X = X.reshape(-1,1)
X = Scaler.fit_transform(X)
X = X.reshape(304,1000,10,6)

In [ ]:
y = y.reshape(-1,1)
for i in range(304):
    if i <= 80:
        y[i][0] = 1
    elif i <= 150:
        y[i][0] = 2
    else:
        y[i][0] = 3

In [ ]:
X_shuffle, y_shuffle = shuffle(X, y)
train_X, train_y = X_shuffle[:263,:,:,:], y_shuffle[:263]
test_X, test_y = X_shuffle[263:, :, :, :], y_shuffle[263:]

In [ ]:
# 自定义BNN层（这一部分若使用GPU训练会存在许多的版本问题，使用CPU问题少一点）
from tensorflow.compat.v1.keras import backend as K
from tensorflow.compat.v1.keras import activations, initializers
from tensorflow.compat.v1.keras.layers import Layer
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.layers import Input
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras import *

def mixture_prior_params(sigma_1, sigma_2, pi, return_sigma=False):
    params = K.variable([sigma_1, sigma_2, pi], name='mixture_prior_params')
    sigma = np.sqrt(pi * sigma_1 ** 2 + (1 - pi) * sigma_2 ** 2)
    return params, sigma
 
def log_mixture_prior_prob(w):
    comp_1_dist = tf.distributions.Normal(0.0, prior_params[0])
    comp_2_dist = tf.distributions.Normal(0.0, prior_params[1])
    comp_1_weight = prior_params[2]    
    return K.log(comp_1_weight * comp_1_dist.prob(w) + (1 - comp_1_weight) * comp_2_dist.prob(w))    
 
# Mixture prior parameters shared across DenseVariational layer instances
prior_params, prior_sigma = mixture_prior_params(sigma_1=1.0, sigma_2=0.1, pi=0.2)
 
class DenseVariational(Layer):
    def __init__(self, output_dim, kl_loss_weight, activation=None, **kwargs):
        self.output_dim = output_dim
        self.kl_loss_weight = kl_loss_weight
        self.activation = activations.get(activation)
        super().__init__(**kwargs)
 
    def build(self, input_shape):  
        self._trainable_weights.append(prior_params) 
 
        self.kernel_mu = self.add_weight(name='kernel_mu', 
                                         shape=(input_shape[1], self.output_dim),
                                         initializer=initializers.normal(stddev=prior_sigma),
                                         trainable=True)
        self.bias_mu = self.add_weight(name='bias_mu', 
                                       shape=(self.output_dim,),
                                       initializer=initializers.normal(stddev=prior_sigma),
                                       trainable=True)
        self.kernel_rho = self.add_weight(name='kernel_rho', 
                                          shape=(input_shape[1], self.output_dim),
                                          initializer=initializers.constant(0.0),
                                          trainable=True)
        self.bias_rho = self.add_weight(name='bias_rho', 
                                        shape=(self.output_dim,),
                                        initializer=initializers.constant(0.0),
                                        trainable=True)
        super().build(input_shape)
 
    def call(self, x):
        kernel_sigma = tf.math.softplus(self.kernel_rho)
        kernel = self.kernel_mu + kernel_sigma * tf.random.normal(self.kernel_mu.shape)
 
        bias_sigma = tf.math.softplus(self.bias_rho)
        bias = self.bias_mu + bias_sigma * tf.random.normal(self.bias_mu.shape)
                
        self.add_loss(self.kl_loss(kernel, self.kernel_mu, kernel_sigma) + 
                      self.kl_loss(bias, self.bias_mu, bias_sigma))
        
        return self.activation(K.dot(x, kernel) + bias)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
    def kl_loss(self, w, mu, sigma):
        variational_dist = tf.distributions.Normal(mu, sigma)
        return kl_loss_weight * K.sum(variational_dist.log_prob(w) - log_mixture_prior_prob(w))

In [ ]:
train_size = train_X.shape[0]
batch_size = 5
num_batches = train_size / batch_size
kl_loss_weight = 1.0 / num_batches

In [ ]:
# CNN+LSTM
model_2 = models.Sequential()
model_2.add(TimeDistributed(Conv1D(64,9, activation='relu',padding='same', input_shape=(1000,10,6))))
model_2.add(TimeDistributed(MaxPooling1D()))
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))
model_2.add(TimeDistributed(MaxPooling1D()))
model_2.add(TimeDistributed(Conv1D(128,9, activation='relu',padding='same')))
model_2.add(TimeDistributed(Flatten())) 
model_2.add(Dropout(0.5)) 
model_2.add(LSTM(units=128, return_sequences=True))
model_2.add(LSTM(units=50, return_sequences=True))
model_2.add(Reshape((50,1000,-1)))
# BNN
model_2.add(DenseVariational(50, kl_loss_weight=kl_loss_weight, activation='relu'))
model_2.add(DenseVariational(20, kl_loss_weight=kl_loss_weight, activation='relu'))
model_2.add(Reshape((50,3,-1)))
model_2.add(DenseVariational(3, kl_loss_weight=kl_loss_weight))

In [ ]:
def neg_log_likelihood(y_obs, y_pred, sigma=1.0):
    dist = tf.distributions.Normal(loc=y_pred, scale=sigma)
    return K.sum(-dist.log_prob(y_obs))
 
model_2.compile(loss=neg_log_likelihood, optimizer=optimizers.Adam(lr=0.001), metrics=['mse'])
model_2.fit(train_X, train_y, batch_size=batch_size, epochs=1500, verbose=0);

# 贝叶斯网络推断

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.factors.discrete import TabularCPD
import pandas as pd
from pgmpy.inference import VariableElimination
from graphviz import Digraph

In [ ]:
battery_model = BayesianModel([('Cell_3', 'Road_2'), 
                              ('Cell_4', 'Road_2'),
                              ('Road_1', 'Battery_Pack'),
                              ('Road_2', 'Battery_Pack')])

In [ ]:
def showBN(model,save=False):
    '''传入BayesianModel对象，调用graphviz绘制结构图，jupyter中可直接显示'''
    node_attr = dict(
     style='filled',
     shape='box',
     align='left',
     fontsize='12',
     ranksep='0.1',
     height='0.2'
    )
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    edges=model.edges()
    for a,b in edges:
        dot.edge(a,b)
    if save:
        dot.view(cleanup=True)
    return dot
showBN(battery_model) 

In [ ]:
#model = BayesianModel([('Cell_3', 'Road_2'), ('Cell_4', 'Road_2')])
#model = BayesianModel([('A', 'C'), ('B', 'C')])

In [ ]:
data = pd.read_csv('E:\\毕业设计\\毕业\\V0\data\\电池组状态数据.csv')
#data = pd.DataFrame(data={'A': [0, 0, 1], 'B': [0, 1, 0], 'C': [1, 1, 0]})

In [ ]:
estimator = BayesianEstimator(model, data)

In [ ]:
cpd_C = estimator.estimate_cpd('Road_2')#, prior_type="dirichlet", pseudo_counts=[1,2])

In [ ]:
print(cpd_C)

In [ ]:
cpd_cell3 = TabularCPD(variable='Cell_3', variable_card=3,
                      values=[[0.6], [0.2], [0.2]])
cpd_cell4 = TabularCPD(variable='Cell_4', variable_card=3,
                       values=[[0.6], [0.2], [0.2]])
cpd_road1 = TabularCPD(variable='Road_1', variable_card=3,
                       values=[[0.9],[0.0],[0.1]])
cpd_road2 = TabularCPD(variable='Road_2', variable_card=3,
                        values=[[0.98, 0.3, 0.2, 0.3, 0.0037, 0.0333, 0.1, 0.0208, 0.0088],
                                [0.01, 0.6, 0.4, 0.6, 0.9370, 0.9333, 0.6, 0.3333, 0.1193],
                                [0.01, 0.1, 0.4, 0.1, 0.0593, 0.0333, 0.3, 0.6458, 0.8719]],
                        evidence=['Cell_3', 'Cell_4'],
                        evidence_card=[3,3])
cpd_pack = TabularCPD(variable='Battery_Pack', variable_card=3,
                      values=[[0.9188, 0.6465, 0.3333, 0.3333, 0.0037, 0.0333, 0.3333, 0.0208, 0.0088],
                              [0.0784, 0.3434, 0.3333, 0.3333, 0.9370, 0.9333, 0.3333, 0.3333, 0.0193],
                              [0.0028, 0.0101, 0.3333, 0.3333, 0.0593, 0.0333, 0.3333, 0.6458, 0.9719]],
                      evidence=['Road_1', 'Road_2'], evidence_card=[3,3])

In [ ]:
battery_model.add_cpds(cpd_cell3, cpd_cell4, cpd_road1, cpd_road2, cpd_pack)

In [ ]:
battery_model.check_model()

In [ ]:
battery_model.get_independencies()

In [ ]:
model_infer = VariableElimination(battery_model)
q = model_infer.query(variables=['Road_2'], evidence={})

In [ ]:
print(q)

In [ ]:
data = pd.read_csv('E:\\毕业设计\\毕业\\V0\data\\电池组状态数据.csv')

In [ ]:
predict_data = data.drop(columns=['Battery_Pack'],axis=1)

In [ ]:
predict_data

In [ ]:
y_pred = battery_model.predict(predict_data)

In [ ]:
y_pred

In [ ]:
print((y_pred['Battery_Pack'] == data['Battery_Pack']).sum()/len(data))